# Example 1: Computation of Energies and Gradients

In this example it is shown, how to run the basic energy and gradient computations. Also, the results are compared to
CASSCF implementation provided by Psi4 package.

Firstly, we specify geometry of the system given in Cartesian coordinates.

In [1]:
geometry = [('N', [0.000000000000, 0.000000000000, 0.000000000000]),
            ('C', [0.000000000000, 0.000000000000, 1.498047000000]),
            ('H', [0.000000000000, -0.938765985000, 2.004775984000]),
            ('H', [0.000000000000, 0.938765985000, 2.004775984000]),
            ('H', [-0.744681452, -0.131307432, -0.634501434])]

Subsequently, we specify the system properties.

In [2]:
n_orbs_active = 2
n_elec_active = 2
charge = 0
multiplicity = 1
basis = 'sto-3g'

Now we construct a `ProblemSet` instance containing the information about our electronic structure problem.

In [3]:
import saoovqe

problem = saoovqe.problem.ProblemSet(geometry=geometry,
                                     charge=charge,
                                     multiplicity=multiplicity,
                                     n_electrons_active=n_elec_active,
                                     n_orbitals_active=n_orbs_active,
                                     basis_name=basis)

   => Loading Basis Set <=

    Name: STO-3G
    Role: ORBITAL
    Keyword: None
    atoms 1   entry N          line    71 file /home/martinb/anaconda3/envs/saoovqe-env/share/psi4/basis/sto-3g.gbs 
    atoms 2   entry C          line    61 file /home/martinb/anaconda3/envs/saoovqe-env/share/psi4/basis/sto-3g.gbs 
    atoms 3-5 entry H          line    19 file /home/martinb/anaconda3/envs/saoovqe-env/share/psi4/basis/sto-3g.gbs 



/home/martinb/anaconda3/envs/saoovqe-env/lib/python3.10/site-packages/qiskit_nature/deprecation.py:297: PauliSumOpDeprecationWarning: PauliSumOp is deprecated as of version 0.6.0 and support for them will be removed no sooner than 3 months after the release. Instead, use SparsePauliOp. You can switch to SparsePauliOp immediately, by setting `qiskit_nature.settings.use_pauli_sum_op` to `False`.
  return func(*args, **kwargs)
2023-07-20 03:06:30,525 - SAOOVQE.logger - INFO - SecondQuantizedProblem was created.


The next one is a set of circuits representing orthogonal states used to construct the whole circuits
representing state vectors later.

In [4]:
initial_circuits = saoovqe.OrthogonalCircuitSet.from_problem_set(n_states=2, problem=problem)

2023-07-20 03:06:30,531 - SAOOVQE.logger - INFO - Circuits representing an orthogonal basis were created.


Another ingredient for statevector circuits constructed later will be an ansatz.

In [5]:
ansatz = saoovqe.Ansatz.from_ProblemSet(ansatz=saoovqe.AnsatzType.GUCCSD,
                                               problem=problem,
                                               repetitions=1,
                                               qubit_mapper=problem.fermionic_mapper)

ic| circuit._excitation_list: [((0,), (1,)), ((2,), (3,)), ((0, 2), (1, 3))]
2023-07-20 03:06:31,382 - SAOOVQE.logger - INFO - Ansatz was created.


Now we can create an instance of our SA-OO-VQE solver. Orbital-optimization doesn't have to be used,
but we do use it here, to compare with CASSCF.

In [6]:
from qiskit.primitives import Estimator

estimator = Estimator()

solver = saoovqe.SAOOVQE(estimator=estimator,
                         initial_circuits=initial_circuits,
                         ansatz=ansatz,
                         problem=problem,
                         orbital_optimization_settings={})

2023-07-20 03:06:31,387 - SAOOVQE.logger - INFO - SAOOVQE was created.


Let's create a numerical optimizer, pass it to our solver and compute our energies now.

In [7]:
from qiskit.algorithms.optimizers import SciPyOptimizer

optimizer = SciPyOptimizer('SLSQP', options={'maxiter': 500, 'ftol': 1e-8})
saoovqe_engs = solver.get_energy(optimizer)

print(saoovqe_engs)

2023-07-20 03:06:31,482 - SAOOVQE.logger - INFO - Computing energies...
2023-07-20 03:06:32,932 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 0.27263779  0.27263052 -0.01551904]
2023-07-20 03:06:32,932 - SAOOVQE.logger - INFO - Starting Orbital-Optimization process...
2023-07-20 03:06:52,816 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 0.19537658  0.19539013 -0.01356976]
2023-07-20 03:06:52,816 - SAOOVQE.logger - INFO - Starting Orbital-Optimization process...
2023-07-20 03:07:05,717 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 0.15572574  0.15571016 -0.01270756]
2023-07-20 03:07:05,717 - SAOOVQE.logger - INFO - Starting Orbital-Optimization process...
2023-07-20 03:07:16,829 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 0.13519     0.13519697 -0.01244268]
2023-07-20 03:07:16,829 - SAOOVQE.logger - INFO - Starting Orbital-Optimization process...
2023-07-20 03:07:25,330 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 

[-92.68211799 -92.63750324]


Also, without further setting things up, we're ready to compute our gradients now!

In [8]:
saoovqe_grads = []
for state_idx in range(2):
    saoovqe_grads.append(list())
    for atom_idx in range(len(geometry)):
        grad = solver.eval_eng_gradient(state_idx, atom_idx)
        print(grad)
        saoovqe_grads[state_idx].append(grad)

2023-07-20 03:08:25,262 - SAOOVQE.logger - INFO - Constructing gradient evaluators...


[-0.22492593 -0.04892123 -0.20123659]
[0.0745678  0.01792576 0.23635004]
[-0.00312258  0.02465681 -0.02605481]
[-0.00628439 -0.02588162 -0.02037615]
[0.15976511 0.03222028 0.01131751]
[-0.04097871 -0.02462531 -0.54292612]
[0.0648765  0.00105069 0.51651813]
[-0.02684153  0.03139331 -0.02809518]
[-0.03878551 -0.03016996 -0.02734765]
[0.04172925 0.02235127 0.08185082]


Perfect! For now, let's set our Psi4 CASSCF solver up. Let's start with re-writing our geometry specification in Psi4-convenient
way and passing it to the toolkit.

In [9]:
import psi4
import numpy as np

def gen_formaldimine_geom_psi4(alpha, phi):
    variables = [1.498047, 1.066797, 0.987109, 118.359375, alpha, phi]

    # Create Z-matrix
    string_geo_dum = '''0 1
                    N
                    C 1 {0}
                    H 2 {1}  1 {3}
                    H 2 {1}  1 {3} 3 180
                    H 1 {2}  2 {4} 3 {5}
                    symmetry c1
                    '''.format(*variables)

    # Convert to Cartesian coordinates
    psi4.core.set_output_file('out.txt', False)
    molecule_dum = psi4.geometry(string_geo_dum)
    molecule_dum.translate(psi4.core.Vector3(-molecule_dum.x(0), -molecule_dum.y(0), -molecule_dum.z(0)))
    mol_geom_dum = molecule_dum.geometry().np * problem.unit_constants['Bohr_to_Angstrom']

    if not np.isclose(mol_geom_dum[1, 1], 0.):
        mol_geom_dum[:, [1, 2]] = mol_geom_dum[:, [2, 1]]
        mol_geom_dum[4, 0] = -mol_geom_dum[4, 0]

    string_geo = ''
    for i, e in enumerate(mol_geom_dum):
        string_geo += f'{molecule_dum.flabel(i)}, {e[0]}, {e[1]}, {e[2]}\n'

    string_geo += 'symmetry c1\n' \
                  'nocom\n' \
                  'noreorient\n'

    return string_geo

psi4.geometry(gen_formaldimine_geom_psi4(130, 80))

Now we'll specify all options for Psi4 CASSCF solver.

In [10]:
n_orbs_frozen = len(problem.frozen_orbitals_indices)

def psi4_casscf():
    psi4.set_options({'basis': basis,
                      'DETCI_FREEZE_CORE': False,
                      'reference': 'RHF',
                      'scf_type': 'pk',
                      'num_roots': len(initial_circuits),
                      'frozen_docc': [0],
                      'restricted_docc': [n_orbs_frozen],
                      'active': [n_orbs_active],
                      'restricted_uocc': [solver.n_mo_optim - n_orbs_frozen - n_orbs_active],
                      'frozen_uocc': [problem.virtual_orbitals_indices[-1] - solver.n_mo_optim + 1],
                      'MAXITER': 1000,
                      'DIIS': False,
                      'D_CONVERGENCE': 1e-6,
                      'E_CONVERGENCE': 1e-6,
                      'S': 0,
                      'avg_states': [0, 1],
                      'avg_weights': [0.5, 0.5]})
    psi4.energy('scf', return_wfn=True)
    psi4.energy('casscf', return_wfn=True)

And run CASSCF preceded by a common SCF computation!

In [11]:
psi4_casscf()

psi4_eng0 = psi4.variable('CI ROOT 0 TOTAL ENERGY')
psi4_eng1 = psi4.variable('CI ROOT 1 TOTAL ENERGY')

Cool! Now we can check, if they're similar to our SA-OO-VQE-obtained energies...

In [12]:
print(psi4_eng0 - saoovqe_engs[0])
print(psi4_eng1 - saoovqe_engs[1])

9.112910021258358e-06
-9.114751094330131e-06


Congratulations! Now it seems, that our energies are really close. Let's compute some gradients
via Psi4. In this case, we'll stick to simple stuff and utilize finite-differences approach.
Specifically, we'll obtain derivatives w.r.t. the $\alpha$ angle. So, first of all, let's generate
shifted geometries.

In [13]:
delta = 1e-5

geom_plus = gen_formaldimine_geom_psi4(alpha=130+(delta/2), phi=80)
geom_minus = gen_formaldimine_geom_psi4(alpha=130-(delta/2),phi=80)

With geometries prepared, we can evaluate the corresponding energies.

In [14]:
psi4.geometry(geom_plus)
psi4_casscf()
psi4_eng0_plus = psi4.variable('CI ROOT 0 TOTAL ENERGY')
psi4_eng1_plus = psi4.variable('CI ROOT 1 TOTAL ENERGY')

psi4.geometry(geom_minus)
psi4_casscf()
psi4_eng0_minus = psi4.variable('CI ROOT 0 TOTAL ENERGY')
psi4_eng1_minus = psi4.variable('CI ROOT 1 TOTAL ENERGY')

And finally, let's evaluate Psi4 gradients w.r.t. $\alpha$.

In [15]:
psi4_grad_0 = (psi4_eng0_plus - psi4_eng0_minus)/delta
psi4_grad_1 = (psi4_eng1_plus - psi4_eng1_minus)/delta

The gradients obtained directly from SA-OO-VQE are evaluated w.r.t. Cartesian coordinates, so we'll transform
these to internal coordinate system to be able to compare the results to each other.

In [16]:
def cartesian_to_inner(vector, R, phi, alpha):
    phi_rad = np.deg2rad(phi)
    alpha_rad = np.deg2rad(alpha)
    v_R = -(vector[0] * np.sin(phi_rad) * np.sin(alpha_rad)) - (vector[1] * np.cos(phi_rad) * np.sin(alpha_rad)) + (vector[2] * np.cos(alpha_rad))
    v_phi = -(vector[0] * R * np.cos(phi_rad) * np.sin(alpha_rad)) + (vector[1] * R * np.sin(phi_rad) * np.sin(alpha_rad))
    v_alpha = -(vector[0] * R * np.sin(phi_rad) * np.cos(alpha_rad)) - (vector[1] * R * np.cos(phi_rad) * np.cos(alpha_rad)) \
              - (vector[2] * R * np.sin(alpha_rad))

    return np.pi * np.array([v_R , v_phi, v_alpha]) / 180

bond_length = np.sqrt(sum(np.array(geometry[4][1])**2))
saoovqe_grad_0_inner = cartesian_to_inner(saoovqe_grads[0][4], bond_length, 80, 130)
saoovqe_grad_1_inner = cartesian_to_inner(saoovqe_grads[1][4], bond_length, 80, 130)

print(saoovqe_grad_0_inner[2])
print(saoovqe_grad_1_inner[2])
print(psi4_grad_0)
print(psi4_grad_1)

0.001654977271923073
-0.0005821615397448378
0.001655618575568951
-0.0005825953053317789


And bingo! Even the gradients seem pretty close, so they're hopefully correct :D